In [5]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Baixar por
# pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

ImportError: cannot import name 'DataLoader' from 'torch.utils.data' (unknown location)

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")


Using cpu device


In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU () ,
        nn.Linear(512,512),
        nn.ReLU (),
        nn.Linear(512,10),
    )
  def forward(self,x):
      x = self.flatten(x)
      logits = self.linear_relu_stack(x)
      return logits

In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU ()
    )
    self.linear_tahn = nn.Sequential(
       nn.Linear(512, 512),
       nn.Relu()
    )
    self.conv = torch.nn.Conv1d(256, 1, 2)
  def forward(self,x):
        print("x.shape: ", x.shape)
        x = self.linear_relu(x)
        print("x.shape: ", x.shape)
        x = self.linear_tahn(x)
        print("x.shape: ", x.shape)
        x = x.reshape((256, 2))
        print("x.shape: ", x.shape)
        logits = self.conv(x)
        print("logits.shape: ", logits.shape)
        return logits

In [ ]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7])


In [ ]:
# Feito em sala

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu = nn.Sequential(
        nn.Linear(300, 512),
        nn.ReLU ()
    )
    self.linear_tahn = nn.Sequential(
       nn.Linear(512, 512),
       nn.ReLU ()
    )
    self.conv = torch.nn.Conv1d(256, 32, 2)
  def forward(self,x):
        print("x.shape: ", x.shape)
        x = self.linear_relu(x)
        print("x.shape: ", x.shape)
        x = self.linear_tahn(x)
        print("x.shape: ", x.shape)
        x = x.reshape((256, 2))
        print("x.shape: ", x.shape)
        logits = self.conv(x)
        print("logits.shape: ", logits.shape)
        return logits

In [ ]:
X = torch.rand(300, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

In [ ]:
from torchtext.vocab import build_vocab_from_iterator
# Build the vocabulary
def build_vocab(texts,tokenizer):
    tokens = []
    for text in texts:
       tokens.extend(tokenizer(text))
    return tokens

train_tokens = build_vocab(train_corpus)
vocab = build_vocab_from_iterator(train_tokens)

NameError: name 'train_corpus' is not defined

In [ ]:
from torch.utils.data import Dataset
class TextDataset(Dataset):
    def __init__(self, file_paths, vocab):
        self.data = self.load_data(file_paths)
        self.vocab = vocab

    def load_data(self, file_paths):
        for file_path in file_paths:
           with open(file_path, 'r', encoding='utf-8') as f:
               tokens = tokenizer(f.read())
        return tokens

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        example = self.data[idx]
        # Convert the example into numerical representations using the vocabulary
        numerical_tokens = [self.vocab[token] for token in example]
        return numerical_tokens


# Create data loaders for the training and validation sets
train_dataset = TextDataset(train_files, vocab)
valid_dataset = TextDataset(valid_files, vocab)

In [ ]:
class TextClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super(TextClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.rnn(embedded)
        last_hidden = output[:, -1, :]
        logits = self.fc(last_hidden)
        return logits

In [ ]:
# Training parameters
vocab_size = len(vocab)
embedding_dim = 100
hidden_dim = 128
num_classes = 10
batch_size = 32
num_epochs = 10
learning_rate = 0.001

# Create the model
model = TextClassifier(vocab_size, embedding_dim, hidden_dim, num_classes)

In [ ]:
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score

#Dataloaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

#Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Iterate over the training data for the specified number of epochs
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    total_samples = 0
    for inputs in train_loader:
        optimizer.zero_grad()
        inputs = torch.LongTensor(inputs)
        targets = inputs.clone()
        outputs = model(inputs)
        loss = criterion(outputs.view(-1, num_classes), targets.view(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * len(inputs)
        total_samples += len(inputs)

    # Evaluate on the validation set after every epoch
    model.eval()
    total_val_loss = 0.0
    total_val_samples = 0
    with torch.no_grad():
        for inputs in valid_loader:
            inputs = torch.LongTensor(inputs)
            targets = inputs.clone()
            outputs = model(inputs)
            val_loss = criterion(outputs.view(-1, num_classes), targets.view(-1))

            total_val_loss += val_loss.item() * len(inputs)
            total_val_samples += len(inputs)

    avg_loss = total_loss / total_samples
    avg_val_loss = total_val_loss / total_val_samples

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_loss:.4f}, Val Loss: {avg_val_loss:.4f}")